# **Лабораторна робота 6: Пошук аномалій та вирішення задачі *anomaly detection* за допомогою бібліотек `scikit-learn`та `PyTorch`**
**Всі завдання виконуються індивідуально. Використання запозиченого коду буде оцінюватись в 0 балів.**

**Лабораторні роботи де в коді буде використаня КИРИЛИЦІ будуть оцінюватись в 20 балів.**

### Мета роботи:
Ознайомитися з основними методами виявлення аномалій, навчитися використовувати бібліотеки `scikit-learn` та `PyTorch` для реалізації алгоритмів пошуку аномалій, проаналізувати ефективність різних методів на реальних наборах даних з Kaggle.


### Опис завдання:

1. **Постановка задачі**:
   Використовуючи один із доступних наборів даних Kaggle (наприклад, *Credit Card Fraud Detection*, *Network Intrusion*, або інші), вам потрібно розв'язати задачу виявлення аномалій. Основна мета — ідентифікувати аномальні записи серед нормальних. Вибраний набір даних повинен містити мітки аномалій для перевірки результатів.

2. **Етапи виконання завдання**:
   - Завантажте та підготуйте набір даних.
   - Проведіть попередню обробку даних (масштабування, заповнення пропущених значень, видалення нерелевантних ознак).
   - Використайте різні методи виявлення аномалій:
     - **Методи з бібліотеки scikit-learn**:
       - Isolation Forest
       - One-Class SVM
       - Local Outlier Factor (LOF)
     - **Методи з використанням PyTorch**:
       - Автоенкодери для виявлення аномалій.
   - Порівняйте отримані результати, обчисліть метрики якості (Precision, Recall, F1-Score).
   - Оцініть, який метод найкраще підходить для вирішення задачі на вашому наборі даних.

### Покрокова інструкція

1. **Підготовка середовища**:
   - Встановіть необхідні бібліотеки:
     ```
     pip install scikit-learn torch pandas numpy matplotlib
     ```

2. **Вибір набору даних з Kaggle**:
   Зареєструйтесь на Kaggle та оберіть один із наборів даних для виявлення аномалій. Наприклад:
   - [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud)
   - [Network Intrusion Detection](https://www.kaggle.com/xyuanh/benchmarking-datasets)

3. **Попередня обробка даних**:
   - Завантажте дані та проведіть їхню початкову обробку.
   - Масштабуйте ознаки за допомогою `StandardScaler` або `MinMaxScaler`.
   - Розділіть дані на навчальну і тестову вибірки.

4. **Методи з бібліотеки `scikit-learn`**:

   - **Isolation Forest**:
     ```
     from sklearn.ensemble import IsolationForest
     ```

   - **One-Class SVM**:
     ```
     from sklearn.svm import OneClassSVM
     ```

   - **Local Outlier Factor**:
     ```
     from sklearn.neighbors import LocalOutlierFactor
     ```

5. **Методи на основі нейронних мереж (PyTorch)**:

   Використайте автоенкодер для пошуку аномалій. Побудуйте нейронну мережу з енкодером і декодером. Під час навчання порівняйте відновлені дані з вхідними та обчисліть помилку. Записи з великою помилкою можуть бути аномаліями.

   - **Реалізація автоенкодера**:
     ```
     import torch
     import torch.nn as nn
     import torch.optim as optim
     ```

6. **Оцінка результатів**:
   Використовуйте метрики оцінки якості:
   - `Precision`, `Recall`, `F1-score`
   ```
   from sklearn.metrics import classification_report
   ```

7. **Звіт**:
   - Поясніть, який метод дав найкращі результати.
   - Проаналізуйте, чому деякі методи працюють краще на вашому наборі даних.
   - Оцініть можливості використання глибоких нейронних мереж (автоенкодерів) для вирішення задачі.


### Результати, які необхідно надати:
1. Код рішення у вигляді Jupyter Notebook з аналізом результатів та поясненнями.


### Дедлайн:
[23 жовтня 23:59]


### Корисні ресурси:
- [Документація PyTorch](https://pytorch.org/docs/stable/index.html)
- [Документація scikit-learn](https://scikit-learn.org/stable/documentation.html)
- [Kaggle Datasets](https://www.kaggle.com/datasets)

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data = pd.read_csv('creditcard.csv')

# вибирає меньшу вибірку данних для швидшого навчання
data_sample = data.sample(frac=0.1, random_state=42)  #10% даних

# маштаб ознак
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_sample.drop('Class', axis=1))

# роздідення на тест і навч вибірки
X_train, X_test, y_train, y_test = train_test_split(data_scaled, data_sample['Class'], test_size=0.2, random_state=42)

# перевірка розміру данних
print(f'Розмір навчальної вибірки: {X_train.shape}')
print(f'Розмір тестової вибірки: {X_test.shape}')


#-------------------------------------------------------------------------------------------------------------#

from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report

# Isolation Forest
iso_forest = IsolationForest(contamination=0.01, random_state=42)
iso_forest.fit(X_train)
y_pred_train_iso = iso_forest.predict(X_train)
y_pred_test_iso = iso_forest.predict(X_test)

# перетворення результ
y_pred_test_iso = np.where(y_pred_test_iso == 1, 0, 1)
print("Isolation Forest")
print(classification_report(y_test, y_pred_test_iso))

# One-Class SVM
svm = OneClassSVM(nu=0.01, kernel="rbf", gamma=0.1)
svm.fit(X_train)
y_pred_train_svm = svm.predict(X_train)
y_pred_test_svm = svm.predict(X_test)

# перетворення результ
y_pred_test_svm = np.where(y_pred_test_svm == 1, 0, 1)
print("One-Class SVM")
print(classification_report(y_test, y_pred_test_svm))

# Local Outlier Factor
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01)
y_pred_train_lof = lof.fit_predict(X_train)
y_pred_test_lof = lof.fit_predict(X_test)

# перетворення результ
y_pred_test_lof = np.where(y_pred_test_lof == 1, 0, 1)
print("Local Outlier Factor")
print(classification_report(y_test, y_pred_test_lof))


#--------------------------------------------------------------------------------#

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# автоенкодер
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# пайторч підготовка
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# ініц моделі
model = Autoencoder(input_dim=X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#автоенкодер навч
num_epochs = 10  # зменьшив кількість епох
for epoch in range(num_epochs):
    for data in train_loader:
        inputs = data[0]
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


#--------------------------------------------------------------------------------#

# обч помилки відновлення для тест данних
with torch.no_grad():
    reconstructions = model(torch.tensor(X_test, dtype=torch.float32))
    loss = torch.mean((reconstructions - torch.tensor(X_test, dtype=torch.float32))**2, dim=1)
    y_pred_test_ae = (loss > torch.quantile(loss, 0.99)).numpy().astype(int)
    
print("Autoencoder")
print(classification_report(y_test, y_pred_test_ae))


#---------------------------------------------------------------------------------#




Розмір навчальної вибірки: (22784, 30)
Розмір тестової вибірки: (5697, 30)
Isolation Forest
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5691
           1       0.07      0.67      0.13         6

    accuracy                           0.99      5697
   macro avg       0.54      0.83      0.56      5697
weighted avg       1.00      0.99      0.99      5697

One-Class SVM
              precision    recall  f1-score   support

           0       1.00      0.88      0.94      5691
           1       0.01      1.00      0.02         6

    accuracy                           0.88      5697
   macro avg       0.50      0.94      0.48      5697
weighted avg       1.00      0.88      0.93      5697

Local Outlier Factor
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5691
           1       0.07      0.67      0.13         6

    accuracy                           0.99      5697
 

На основі отриманих результатів, найкращим методом для виявлення аномалій у даному наборі даних є Isolation Forest. Він показав збалансовані результати між precision та recall для класу 1.